In [0]:
# A Machine Learning Project that takes an RGB image of a number in sign language
# and outputs its corresponding value using Neural Networks

In [0]:
! git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset

fatal: destination path 'Sign-Language-Digits-Dataset' already exists and is not an empty directory.


In [0]:
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from os import listdir
from scipy.misc import *
import imageio
from skimage import transform,io
import tensorflow as tf
from tensorflow import keras
from numpy import array
from sklearn.model_selection import KFold

# Settings:
grayscale_images = True
num_class = 10
test_size = 0.2


In [0]:
def get_img(data_path):
    img = io.imread(data_path)
    return img

def print_img(img):
    imgplot = plt.imshow(img)
    plt.show()

In [0]:
def get_dataset(dataset_path='./Dataset/'):
    # Getting all data from data path:
    try:
        print("in try")
        X = np.load('npy_dataset/X.npy')
        Y = np.load('npy_dataset/Y.npy')
    except:
        print("inside")
        labels = listdir(dataset_path) # Getting labels
        X = []
        Y = []
        channel_sum = np.zeros(3)
        pixel_num = 0
        for i, label in enumerate(labels):
            datas_path = dataset_path+'/'+label
            for data in listdir(datas_path):
                img = get_img(datas_path+'/'+data)
                
                if(img.shape[0] > 100):
                  X.append(X[-1])
                  Y.append(Y[-1])
                else:
                  X.append(img)
                  Y.append(int(label))

                pixel_num += 100*100
                channel_sum += np.sum(img, axis=(0, 1))
        # Create dateset:
        print_img(X[200])
        print(Y[1])

        # calculate average
        
        rgb_mean = channel_sum / pixel_num
        
        for i in range(len(X)):
          X[i] = 1-np.array(X[i]).astype('float64')
          X[i] -= rgb_mean

        for i in range(len(X)):
            X[i] = X[i]/255.
        Y = np.array(Y).astype('float64')

        # print(X)
        if not os.path.exists('npy_dataset/'):
            os.makedirs('npy_dataset/')
        np.save('npy_dataset/X.npy', X)
        np.save('npy_dataset/Y.npy', Y)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [0]:
train_x, val_x, test_x, train_y, val_y, test_y = get_dataset('Sign-Language-Digits-Dataset/Dataset')

in try


In [0]:
train_x = np.array(train_x)
print(train_x.shape)
train_x = np.stack(train_x)
print(train_x.shape)

(1236, 100, 100, 3)
(1236, 100, 100, 3)


In [0]:
print(train_x.shape)
print(train_y.shape)
print(np.asarray(train_y).shape)

(1236, 100, 100, 3)
(1236,)
(1236,)


In [0]:
test_x = np.array(test_x)
test_x = np.stack(test_x)
test_x.shape

(413, 100, 100, 3)

In [0]:
def train_test_model(layers, train_x, val_x, train_y, val_y):
  cnn = keras.Sequential()
  # using conv layer on input
  cnn.add(keras.layers.Conv2D(filters=32, 
                kernel_size=(2,2), 
                strides=(1,1),
                padding='same',
                input_shape=(100,100,3),
                data_format='channels_last'))
  cnn.add(keras.layers.Activation('relu'))

  cnn.add(keras.layers.MaxPooling2D(pool_size=(2,2),
                      strides=2))

  # number of conv and pooling layers determined by function parameter
  for i in range(int(layers)):
    cnn.add(keras.layers.Conv2D(filters=64,
                  kernel_size=(2,2),
                  strides=(1,1),
                  padding='valid'))
    cnn.add(keras.layers.Activation('relu'))

    cnn.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2))

  cnn.add(keras.layers.Flatten())
  cnn.add(keras.layers.Dense(64))
  cnn.add(keras.layers.Activation('relu'))
  cnn.add(keras.layers.Dropout(0.25))

  # using softmax in output layer for classification
  cnn.add(keras.layers.Dense(10, activation='softmax'))

  cnn.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

  cnn.fit(train_x, train_y, epochs=1)
  
  val_test_loss, val_test_acc = cnn.evaluate(val_x,  val_y, verbose=2)
  print('\nvalidation accuracy for model (', layers, ' layers', '):', val_test_acc * 100.0)
  print('==================================================================================')

  test_loss, test_acc = cnn.evaluate(test_x,  test_y, verbose=2)
  print('\nTest accuracy for model (', layers, ' layers', '):', test_acc * 100.0)
  print('==================================================================================')

  return test_acc

In [0]:
avg1 = 0
avg2 = 0
avg3 = 0
avg4 = 0
kfold = KFold(7, True, 1)
co = 1
for train, test in kfold.split(train_x):
  print("Fold number: ", co)
  co += 1
  avg1 += train_test_model(1, train_x[train], train_x[test], train_y[train], train_y[test])

Fold number:  1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1059 samples
1059/1059 [==============================] - 9s 9ms/sample - loss: 3.8864 - acc: 0.0963
177/177 - 1s - loss: 2.3032 - acc: 0.1073

validation accuracy for model ( 1  layers ): 10.734463483095169
413/413 - 1s - loss: 2.3023 - acc: 0.1114

Test accuracy for model ( 1  layers ): 11.138014495372772
Fold number:  2
Train on 1059 samples
1059/1059 [==============================] - 9s 9ms/sample - loss: 3.9320 - acc: 0.0925
177/177 - 1s - loss: 2.3029 - acc: 0.0960

validation accuracy for model ( 1  layers ): 9.604519605636597
413/413 - 1s - loss: 2.3032 - acc: 0.0775

Test accuracy for model ( 1  layers ): 7.748184353113174
Fold number:  3
Train on 1059 samples
1059/1059 [==============================] - 9s 9ms/sample - loss: 3.3498 - acc: 0.1039
177/177 - 1s - loss: 2.3027 - acc: 0.0904

validation accuracy for model ( 1  layers ): 9.03954803943634
413/413 - 1s - loss: 2

In [0]:
for train, test in kfold.split(train_x):
  print("Fold number: ", co)
  co += 1
  avg2 += train_test_model(2, train_x[train], train_x[test], train_y[train], train_y[test])

Fold number:  8
Train on 1059 samples
1059/1059 [==============================] - 10s 10ms/sample - loss: 2.4692 - acc: 0.0963
177/177 - 1s - loss: 2.3128 - acc: 0.1073

validation accuracy for model ( 2  layers ): 10.734463483095169
413/413 - 1s - loss: 2.3088 - acc: 0.1114

Test accuracy for model ( 2  layers ): 11.138014495372772
Fold number:  9
Train on 1059 samples
1059/1059 [==============================] - 10s 10ms/sample - loss: 2.4861 - acc: 0.1237
177/177 - 1s - loss: 2.3034 - acc: 0.0960

validation accuracy for model ( 2  layers ): 9.604519605636597
413/413 - 1s - loss: 2.3158 - acc: 0.0775

Test accuracy for model ( 2  layers ): 7.748184353113174
Fold number:  10
Train on 1059 samples
1059/1059 [==============================] - 10s 10ms/sample - loss: 2.3928 - acc: 0.1058
177/177 - 1s - loss: 2.2999 - acc: 0.1243

validation accuracy for model ( 2  layers ): 12.429378181695938
413/413 - 1s - loss: 2.3071 - acc: 0.0944

Test accuracy for model ( 2  layers ): 9.4430990517

In [0]:
for train, test in kfold.split(train_x):
  print("Fold number: ", co)
  co += 1
  avg3 += train_test_model(3, train_x[train], train_x[test], train_y[train], train_y[test])

Fold number:  15
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3155 - acc: 0.0982
177/177 - 1s - loss: 2.3142 - acc: 0.0904

validation accuracy for model ( 3  layers ): 9.03954803943634
413/413 - 1s - loss: 2.3050 - acc: 0.1404

Test accuracy for model ( 3  layers ): 14.04358297586441
Fold number:  16
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3171 - acc: 0.0954
177/177 - 1s - loss: 2.3074 - acc: 0.0678

validation accuracy for model ( 3  layers ): 6.779661029577255
413/413 - 1s - loss: 2.3038 - acc: 0.0775

Test accuracy for model ( 3  layers ): 7.748184353113174
Fold number:  17
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3562 - acc: 0.0925
177/177 - 1s - loss: 2.3019 - acc: 0.1243

validation accuracy for model ( 3  layers ): 12.429378181695938
413/413 - 1s - loss: 2.3080 - acc: 0.0944

Test accuracy for model ( 3  layers ): 9.44309905171

In [0]:
for train, test in kfold.split(train_x):
  print("Fold number: ", co)
  co += 1
  avg4 += train_test_model(4, train_x[train], train_x[test], train_y[train], train_y[test])

Fold number:  22
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3095 - acc: 0.0954
177/177 - 1s - loss: 2.3186 - acc: 0.0904

validation accuracy for model ( 4  layers ): 9.03954803943634
413/413 - 1s - loss: 2.3174 - acc: 0.0775

Test accuracy for model ( 4  layers ): 7.748184353113174
Fold number:  23
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3214 - acc: 0.1086
177/177 - 1s - loss: 2.3168 - acc: 0.0960

validation accuracy for model ( 4  layers ): 9.604519605636597
413/413 - 1s - loss: 2.3094 - acc: 0.0775

Test accuracy for model ( 4  layers ): 7.748184353113174
Fold number:  24
Train on 1059 samples
1059/1059 [==============================] - 11s 10ms/sample - loss: 2.3209 - acc: 0.1001
177/177 - 1s - loss: 2.3350 - acc: 0.1243

validation accuracy for model ( 4  layers ): 12.429378181695938
413/413 - 1s - loss: 2.3549 - acc: 0.0944

Test accuracy for model ( 4  layers ): 9.44309905171

In [0]:
print('model 1 avg test accuracy: ', (avg1 / 7) * 100)
print('model 2 avg test accuracy: ', (avg2 / 7) * 100)
print('model 3 avg test accuracy: ', (avg3 / 7) * 100)
print('model 4 avg test accuracy: ', (avg4 / 7) * 100)

model 1 avg test accuracy:  9.92736081991877
model 2 avg test accuracy:  9.443099371024541
model 3 avg test accuracy:  9.13178835596357
model 4 avg test accuracy:  9.443099371024541
